# **Wine Quality Prediction**

В этом соревновании нам предстоит предсказать качество вина по заданным параметрам  

# **Import**

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 

import warnings
warnings.filterwarnings('ignore')

from itertools import combinations
from scipy.stats import ttest_ind

from sklearn.feature_selection import f_classif
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsRegressor as kNN
import xgboost as xgb
from sklearn.svm import SVR
from mlxtend.regressor import StackingCVRegressor
from sklearn.neural_network import MLPRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [2]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [3]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# **DATA**

In [4]:
data = pd.read_csv("../input/wine-quality-dataset/WineQT.csv")

Посмотрим на датасет

In [5]:
data.head()

In [6]:
data.info()

Этот датафрейм содержит следующие столбцы:

1 - фиксированная кислотность\
2 - летучая кислотность\
3 - лимонная кислота\
4 - остаточный сахар\
5 - хлориды\
6 - свободный диоксид серы\
7 - общий диоксид серы\
8 - плотность\
9 - pH\
10 - сульфаты\
11 - спирт\
12 - качество (целевая переменная, оценка от 0 до 10)\
13 - id

In [7]:
data = data.drop(['Id'], axis=1) #удалим столбец id
print(data.isna().sum()) #проверим на пропущенные значения

Как видим все значения числовые, пропусков нет

# **Feature engineering**

Добавим новый признак, отношение free sulfur dioxide к total sulfur dioxide

In [8]:
data ['division sulfur dioxide'] = data ['free sulfur dioxide'] / data ['total sulfur dioxide']

# **EDA**

In [9]:
plt.rcParams['figure.figsize'] = (10,5)
data['quality'].hist(bins=10)

Видим нормальное расспределение, у подовляющего большинства представленных образцов средние оценки

Посмотрим на матрицу корреляции

In [10]:
sns.heatmap(data.corr(), annot=True)

Как видно из матрицы корреляции сильно скоррелированных признаков нет

Объеденим все признаки 

In [11]:
num_cols = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 
            'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 
            'pH', 'sulphates', 'alcohol', 'division sulfur dioxide']

In [12]:
fig, axes = plt.subplots(4, 3, figsize=(15, 15))
axes = axes.flatten()
for i in range(len(num_cols)):
    sns.boxplot(x="quality", y=num_cols[i], data=data, ax=axes[i], showfliers=False)

**Значимость переменных**

In [13]:
imp_num = pd.Series(f_classif(data [num_cols], data['quality'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh', title='Значимость числовых переменных')

Из графика видно, что наиболее значимый признак % спирта в вине

In [14]:
def get_stat_dif(column):
    cols = data.loc[:, column].value_counts().index[:10]
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(data.loc[data.loc[:, column] == comb[0], 'quality'], 
                        data.loc[data.loc[:, column] == comb[1], 'quality']).pvalue \
            <= 0.05/len(combinations_all): # Учли поправку Бонферони
            print('Найдены статистически значимые различия для колонки', column)
            break

In [15]:
for col in num_cols:
    get_stat_dif(col)

Только три параметра серьезно отличаются

# **Data Preprocessing**

In [16]:
# Поскольку в данных выбросы, воспользуемся RobustScaler
scaler = RobustScaler()

data[num_cols] = scaler.fit_transform(data[num_cols].values)

In [17]:
#data = data.loc[:, ['volatile acidity', 'citric acid', 'alcohol', 'quality']]

In [18]:
# Воспользуемся специальной функцией train_test_split для разбивки тестовых данных
X=data.drop(['quality'],axis='columns')
y=data['quality']

In [19]:
#X = X.values

In [20]:
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [21]:
#kf = KFold(random_state=RANDOM_SEED, shuffle=True)

In [22]:
#for train_index, test_index in kf.split(X):
#    print("TRAIN:", train_index, "TEST:", test_index)

In [23]:
#X_train, X_test = X[train_index], X[test_index]

In [24]:
#y_train, y_test = y[train_index], y[test_index]

In [25]:
# проверяем
data.shape, X.shape, X_train.shape, X_test.shape

При крос валидации значение метрик хуже, вернул train_test_split

# **Model**

**Linear Regression**

In [26]:
# Создаём модель 
model=LinearRegression()

In [27]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train,y_train)

In [28]:
# Используем обученную модель для предсказания оценки вина в тестовой выборке.
# Предсказанные значения записываем в переменную predictions
predictions=model.predict(X_test)

In [29]:
# Сравниваем предсказанные значения (predictions) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', mean_absolute_error(y_test, predictions))

**Decision Tree Regressor**

In [30]:
model_2=DecisionTreeRegressor(random_state=RANDOM_SEED)

In [31]:
model_2.fit(X_train,y_train)

In [32]:
predictions_2=model_2.predict(X_test)

In [33]:
print('MAE:', mean_absolute_error(y_test, predictions_2))

Значение метрики улучшилось

**Random Forest Regressor**

In [34]:
model_3=RandomForestRegressor(random_state=RANDOM_SEED)

In [35]:
model_3.fit(X_train,y_train)

In [36]:
predictions_3=model_3.predict(X_test)

In [37]:
print('MAE:', mean_absolute_error(y_test, predictions_3))

Значение метрики лучше чем Linear Regression и Decision Tree Regressor

Подберем гиперпараметры

In [38]:
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [39]:
model_4 = RandomForestRegressor(random_state=RANDOM_SEED)
model_4_random = RandomizedSearchCV(estimator=model_4, param_distributions=random_grid, n_iter=100, 
                               cv=3, verbose=2, random_state=RANDOM_SEED, n_jobs=-1)
model_4_random.fit(X_train, y_train)

Давайте посмотрим, какие гиперпараметры нам предлагают как оптимальные:

In [40]:
model_4_random.best_params_

In [41]:
model_4 = model_4_random.best_estimator_

In [42]:
model_4.fit(X_train, y_train)

In [43]:
predictions_4=model_4.predict(X_test)

In [44]:
print('MAE:', mean_absolute_error(y_test, predictions_4))

Значение метрики улучшилось

# **Стекинг**

Отличный вариант нашел тут https://www.kaggle.com/unfashionable/linear-stacking-top20-run-time-1-min

In [45]:
r1 = kNN()
r2 = Ridge()
r2_1 = Ridge(alpha=0.005)
r2_2 = Ridge(alpha=5)

r3 = RandomForestRegressor(n_estimators=50,n_jobs=-1)
r4 = Lasso(alpha=0.0005)
r4_1 = Lasso()
r4_2 = Lasso(alpha=5)
rx = xgb.XGBRegressor(n_jobs=-1)
r5 = SVR()
r6 = LinearRegression()

regression_stacker = StackingCVRegressor(regressors = [r1,r2,r3,r4,r5,r2_1,r2_2,r4_1,r4_2,rx],meta_regressor = r6, cv=3)
regression_stacker.fit(X_train, y_train)
predictions_5 = regression_stacker.predict(X_test)

In [46]:
print('MAE:', mean_absolute_error(y_test, predictions_5))

Значение метрики не улучшилось

**MLPRegressor**

In [47]:
model_6 = MLPRegressor(random_state=RANDOM_SEED)

In [48]:
model_6.fit(X_train,y_train)

In [49]:
predictions_6=model_6.predict(X_test)

In [50]:
print('MAE:', mean_absolute_error(y_test, predictions_6))

Это наилучший показатель метрики

**LGBMRegressor**

In [51]:
model_7 = LGBMRegressor(random_state=RANDOM_SEED)

In [52]:
model_7.fit(X_train,y_train)

In [53]:
predictions_7=model_7.predict(X_test)

In [54]:
print('MAE:', mean_absolute_error(y_test, predictions_7))

**CatBoostRegressor**

In [55]:
model_8 = model = CatBoostRegressor(iterations = 5000,
                          learning_rate = 0.1,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAE'
                         )
model_8.fit(X_train, y_train,
         eval_set=(X_test, y_test),
         verbose_eval=100,
         use_best_model=True,
         plot=True
         )

In [56]:
predictions_8=model_8.predict(X_test)

In [57]:
print('MAE:', mean_absolute_error(y_test, predictions_8))

**LAMA**

In [58]:
# Install base functionality:

!pip install -U lightautoml